# Horse Racing Prediction by Deep Learning

This AI Horse racing prediction is using a 3-layer Neural Network to predict the finishing position of each horse in a race. The data used to train this model is obtained from the Hong Kong Jockey Club (HKJC). The model is built and trained by Tensorflow. Supervised learning is used in this project to classify the expected finishing position of the horses.

<H2> Part 1: Data Input and Preprocessing </H2>

In this part of the program, we will import the data obtained from the HKJC. First of all, the following features were selected based on my past horse picking experience, namely:

- position: The starting position of the horse. If the position is "1", it indicates the closest position to the hurdle and should be benficial in non-straight race courses.

- load: This is the loading of the horse in pounds. Maximum is 133.

- ON odds: This is the overnight odds of the horse provided by the HKJC.

- odds: This is the odds of the horse 15 min before the race.

- class: This is the class of the case. It is common to all horses in a race except special races.

- num horses: This is the number of horses participated in the race.


In [1]:
#Loading the data and preprocessing it.

import pandas as pd
import tensorflow as tf
tf.__version__

PATH_TRAINING_DATA = 'training_data/horse_data_train_test.csv'

dataset = pd.read_csv(PATH_TRAINING_DATA)
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0, shuffle=False)


In [2]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train) #use "fit_transform" for training data
X_test = sc.transform(X_test)       #use "transform" for testing data

y_train = tf.keras.utils.to_categorical((y_train-1), 14)
y_test = tf.keras.utils.to_categorical((y_test-1), 14)

print(X_train)

[[-0.12704156  0.81437749 -0.43715443 -0.66687959 -0.62680641 -1.26818212]
 [-0.62381396 -0.62792474 -0.75995163 -0.87541498 -0.62680641 -1.26818212]
 [ 0.61811703 -1.0212799  -0.5613072  -0.69468431 -0.62680641 -1.26818212]
 ...
 [ 0.36973083  0.94549588 -0.53026901 -0.30541825 -1.83071653  0.5566608 ]
 [ 1.61166183  1.86332458  2.13901555  1.75213092 -3.03462664  0.5566608 ]
 [ 0.86650323 -0.23456959 -0.7382249  -0.8837564  -1.83071653  0.5566608 ]]


<H2> Part 2: Building the Model </H2>

We will construct a basic 3-layer model: one input layer, one output layer and one hidden layer.
I have tried relu and sigmoid as the activation, and found that sigmoid seems to be a better choice for this project.


In [3]:
# Initializing the ANN
model = tf.keras.models.Sequential()

# Adding the input layer and the first layer
# We have six features now, and one bias term, so the input layer has a size of 6+1
model.add(tf.keras.layers.Dense(units=6, activation='sigmoid'))

# Adding the input layer and the hidden layer
# We set the size of the hidden layer to be the mean of input and output later, i.e. 10
#model.add(tf.keras.layers.Dense(units=6, activation='sigmoid')) 
model.add(tf.keras.layers.Dense(units=10, activation='sigmoid')) 

# Adding a drop out layer
model.add(tf.keras.layers.Dropout(0.2))

#model.add(tf.keras.layers.Dense(units=6, activation='sigmoid')) 
#model.add(tf.keras.layers.Dropout(0.2))

# Adding the output layer
# We have 14 outputs, so the output later has a size of 14
model.add(tf.keras.layers.Dense(units=14, activation='softmax'))

# Compiling the model
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

<H2> Part 3a: [Optional] Training and saving a new Model </H2>

In this section, we would either traing the model from scratch or just load a pre-trained model that is shipped with this Jupyter notebook.

- If the training data has been changed or the model has been renewed, go to train a new model in code cell 3a)
- If you want to save some time, just skip cell 3a) and load a pre-trained model in code cell 3b)


In [4]:
#This is code cell 3a) that trains a model from scratch and then saves it. Please note that it takes hours to train!

import tensorflow as tf
import datetime, os

%load_ext tensorboard

logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

print("rows in X_train = " + str(X_train.shape[0]) )
print("rows in y_train = " + str(y_train.shape[0]) )

print("Training model...")
model.fit(x=X_train, 
          y=y_train, 
          batch_size = 14, 
          epochs = 10000, #30000 #50000 #20000
          validation_data=(X_test, y_test), 
          callbacks=[tensorboard_callback])

# save the model for later use
print("Saving model...")
model.save('saved_model/my_model')

print("Model saved!")

rows in X_train = 436
rows in y_train = 436
Training model...
Epoch 1/10000
32/32 [==============================] - 3s 33ms/step - loss: 2.7507 - accuracy: 0.0849 - val_loss: 2.7102 - val_accuracy: 0.0455
Epoch 2/10000
32/32 [==============================] - 0s 5ms/step - loss: 2.7140 - accuracy: 0.0688 - val_loss: 2.6870 - val_accuracy: 0.0455
Epoch 3/10000
32/32 [==============================] - 0s 5ms/step - loss: 2.6911 - accuracy: 0.0665 - val_loss: 2.6718 - val_accuracy: 0.0455
Epoch 4/10000
32/32 [==============================] - 0s 5ms/step - loss: 2.6647 - accuracy: 0.0963 - val_loss: 2.6608 - val_accuracy: 0.0455
Epoch 5/10000
32/32 [==============================] - 0s 5ms/step - loss: 2.6780 - accuracy: 0.0757 - val_loss: 2.6528 - val_accuracy: 0.0636
Epoch 6/10000
32/32 [==============================] - 0s 5ms/step - loss: 2.6742 - accuracy: 0.0642 - val_loss: 2.6480 - val_accuracy: 0.0455
Epoch 7/10000
32/32 [==============================] - 0s 5ms/step - loss: 2.66

Epoch 114/10000
32/32 [==============================] - 0s 4ms/step - loss: 2.3809 - accuracy: 0.1399 - val_loss: 2.3464 - val_accuracy: 0.1364
Epoch 115/10000
32/32 [==============================] - 0s 4ms/step - loss: 2.3574 - accuracy: 0.1743 - val_loss: 2.3467 - val_accuracy: 0.1273
Epoch 116/10000
32/32 [==============================] - 0s 4ms/step - loss: 2.3792 - accuracy: 0.1468 - val_loss: 2.3469 - val_accuracy: 0.1273
Epoch 117/10000
32/32 [==============================] - 0s 5ms/step - loss: 2.3546 - accuracy: 0.1674 - val_loss: 2.3462 - val_accuracy: 0.1364
Epoch 118/10000
32/32 [==============================] - 0s 5ms/step - loss: 2.3683 - accuracy: 0.1353 - val_loss: 2.3453 - val_accuracy: 0.1364
Epoch 119/10000
32/32 [==============================] - 0s 5ms/step - loss: 2.3631 - accuracy: 0.1445 - val_loss: 2.3445 - val_accuracy: 0.1182
Epoch 120/10000
32/32 [==============================] - 0s 4ms/step - loss: 2.3524 - accuracy: 0.1766 - val_loss: 2.3445 - val_ac

32/32 [==============================] - 0s 5ms/step - loss: 2.3115 - accuracy: 0.1537 - val_loss: 2.3050 - val_accuracy: 0.1636
Epoch 227/10000
32/32 [==============================] - 0s 5ms/step - loss: 2.3026 - accuracy: 0.1720 - val_loss: 2.3051 - val_accuracy: 0.1455
Epoch 228/10000
32/32 [==============================] - 0s 5ms/step - loss: 2.3215 - accuracy: 0.1583 - val_loss: 2.3052 - val_accuracy: 0.1636
Epoch 229/10000
32/32 [==============================] - 0s 5ms/step - loss: 2.2921 - accuracy: 0.1697 - val_loss: 2.3049 - val_accuracy: 0.1455
Epoch 230/10000
32/32 [==============================] - 0s 5ms/step - loss: 2.3047 - accuracy: 0.1445 - val_loss: 2.3045 - val_accuracy: 0.1545
Epoch 231/10000
32/32 [==============================] - 0s 6ms/step - loss: 2.3111 - accuracy: 0.1674 - val_loss: 2.3037 - val_accuracy: 0.1455
Epoch 232/10000
32/32 [==============================] - 0s 5ms/step - loss: 2.3021 - accuracy: 0.1927 - val_loss: 2.3041 - val_accuracy: 0.1364
E

32/32 [==============================] - 0s 5ms/step - loss: 2.2497 - accuracy: 0.1835 - val_loss: 2.3130 - val_accuracy: 0.1273
Epoch 339/10000
32/32 [==============================] - 0s 4ms/step - loss: 2.2384 - accuracy: 0.1995 - val_loss: 2.3146 - val_accuracy: 0.1273
Epoch 340/10000
32/32 [==============================] - 0s 4ms/step - loss: 2.2534 - accuracy: 0.1766 - val_loss: 2.3137 - val_accuracy: 0.1364
Epoch 341/10000
32/32 [==============================] - 0s 4ms/step - loss: 2.2429 - accuracy: 0.1766 - val_loss: 2.3143 - val_accuracy: 0.1364
Epoch 342/10000
32/32 [==============================] - 0s 4ms/step - loss: 2.2419 - accuracy: 0.1950 - val_loss: 2.3135 - val_accuracy: 0.1182
Epoch 343/10000
32/32 [==============================] - 0s 4ms/step - loss: 2.2500 - accuracy: 0.2064 - val_loss: 2.3132 - val_accuracy: 0.1182
Epoch 344/10000
32/32 [==============================] - 0s 4ms/step - loss: 2.2466 - accuracy: 0.1950 - val_loss: 2.3126 - val_accuracy: 0.1182
E

32/32 [==============================] - 0s 4ms/step - loss: 2.2341 - accuracy: 0.1743 - val_loss: 2.3243 - val_accuracy: 0.1273
Epoch 451/10000
32/32 [==============================] - 0s 4ms/step - loss: 2.2246 - accuracy: 0.1881 - val_loss: 2.3255 - val_accuracy: 0.1364
Epoch 452/10000
32/32 [==============================] - 0s 5ms/step - loss: 2.2104 - accuracy: 0.2041 - val_loss: 2.3282 - val_accuracy: 0.1182
Epoch 453/10000
32/32 [==============================] - 0s 4ms/step - loss: 2.2425 - accuracy: 0.1881 - val_loss: 2.3270 - val_accuracy: 0.1273
Epoch 454/10000
32/32 [==============================] - 0s 5ms/step - loss: 2.1941 - accuracy: 0.2064 - val_loss: 2.3272 - val_accuracy: 0.1273
Epoch 455/10000
32/32 [==============================] - 0s 5ms/step - loss: 2.2308 - accuracy: 0.1927 - val_loss: 2.3268 - val_accuracy: 0.1364
Epoch 456/10000
32/32 [==============================] - 0s 5ms/step - loss: 2.2232 - accuracy: 0.1904 - val_loss: 2.3255 - val_accuracy: 0.1364
E

...

32/32 [==============================] - 0s 4ms/step - loss: 1.9743 - accuracy: 0.3119 - val_loss: 3.9134 - val_accuracy: 0.2636
Epoch 9467/10000
32/32 [==============================] - 0s 4ms/step - loss: 1.9463 - accuracy: 0.3073 - val_loss: 3.9028 - val_accuracy: 0.2636
Epoch 9468/10000
32/32 [==============================] - 0s 4ms/step - loss: 1.9227 - accuracy: 0.3005 - val_loss: 3.8913 - val_accuracy: 0.2636
Epoch 9469/10000
32/32 [==============================] - 0s 4ms/step - loss: 1.9472 - accuracy: 0.3073 - val_loss: 3.8771 - val_accuracy: 0.2636
Epoch 9470/10000
32/32 [==============================] - 0s 4ms/step - loss: 1.9491 - accuracy: 0.3188 - val_loss: 3.8787 - val_accuracy: 0.2636
Epoch 9471/10000
32/32 [==============================] - 0s 4ms/step - loss: 1.9227 - accuracy: 0.3096 - val_loss: 3.8806 - val_accuracy: 0.2636
Epoch 9472/10000
32/32 [==============================] - 0s 4ms/step - loss: 1.9096 - accuracy: 0.3005 - val_loss: 3.9078 - val_accuracy: 0.

32/32 [==============================] - 0s 4ms/step - loss: 1.9167 - accuracy: 0.3257 - val_loss: 3.9848 - val_accuracy: 0.2545
Epoch 9579/10000
32/32 [==============================] - 0s 4ms/step - loss: 1.9759 - accuracy: 0.3073 - val_loss: 3.9737 - val_accuracy: 0.2545
Epoch 9580/10000
32/32 [==============================] - 0s 4ms/step - loss: 1.9239 - accuracy: 0.3165 - val_loss: 3.9760 - val_accuracy: 0.2545
Epoch 9581/10000
32/32 [==============================] - 0s 4ms/step - loss: 1.9452 - accuracy: 0.3303 - val_loss: 3.9881 - val_accuracy: 0.2545
Epoch 9582/10000
32/32 [==============================] - 0s 4ms/step - loss: 1.9820 - accuracy: 0.3142 - val_loss: 3.9693 - val_accuracy: 0.2545
Epoch 9583/10000
32/32 [==============================] - 0s 4ms/step - loss: 1.9204 - accuracy: 0.3303 - val_loss: 3.9798 - val_accuracy: 0.2545
Epoch 9584/10000
32/32 [==============================] - 0s 4ms/step - loss: 1.9461 - accuracy: 0.3096 - val_loss: 3.9825 - val_accuracy: 0.

32/32 [==============================] - 0s 4ms/step - loss: 1.9251 - accuracy: 0.3188 - val_loss: 3.9981 - val_accuracy: 0.2636
Epoch 9691/10000
32/32 [==============================] - 0s 4ms/step - loss: 1.9653 - accuracy: 0.2890 - val_loss: 3.9961 - val_accuracy: 0.2636
Epoch 9692/10000
32/32 [==============================] - 0s 4ms/step - loss: 1.9472 - accuracy: 0.3073 - val_loss: 3.9943 - val_accuracy: 0.2636
Epoch 9693/10000
32/32 [==============================] - 0s 4ms/step - loss: 1.9414 - accuracy: 0.3142 - val_loss: 4.0062 - val_accuracy: 0.2636
Epoch 9694/10000
32/32 [==============================] - 0s 4ms/step - loss: 1.9662 - accuracy: 0.3096 - val_loss: 3.9997 - val_accuracy: 0.2545
Epoch 9695/10000
32/32 [==============================] - 0s 4ms/step - loss: 1.9359 - accuracy: 0.3188 - val_loss: 4.0055 - val_accuracy: 0.2636
Epoch 9696/10000
32/32 [==============================] - 0s 4ms/step - loss: 1.9365 - accuracy: 0.3073 - val_loss: 3.9839 - val_accuracy: 0.

32/32 [==============================] - 0s 4ms/step - loss: 1.9226 - accuracy: 0.3349 - val_loss: 4.0275 - val_accuracy: 0.2545
Epoch 9803/10000
32/32 [==============================] - 0s 4ms/step - loss: 1.9508 - accuracy: 0.3050 - val_loss: 4.0405 - val_accuracy: 0.2545
Epoch 9804/10000
32/32 [==============================] - 0s 4ms/step - loss: 1.9613 - accuracy: 0.3188 - val_loss: 4.0418 - val_accuracy: 0.2545
Epoch 9805/10000
32/32 [==============================] - 0s 4ms/step - loss: 1.9227 - accuracy: 0.3188 - val_loss: 4.0446 - val_accuracy: 0.2545
Epoch 9806/10000
32/32 [==============================] - 0s 4ms/step - loss: 1.9204 - accuracy: 0.3142 - val_loss: 4.0453 - val_accuracy: 0.2545
Epoch 9807/10000
32/32 [==============================] - 0s 4ms/step - loss: 1.9520 - accuracy: 0.3119 - val_loss: 4.0468 - val_accuracy: 0.2545
Epoch 9808/10000
32/32 [==============================] - 0s 3ms/step - loss: 1.9459 - accuracy: 0.3188 - val_loss: 4.0259 - val_accuracy: 0.

32/32 [==============================] - 0s 4ms/step - loss: 1.9062 - accuracy: 0.3257 - val_loss: 4.0730 - val_accuracy: 0.2545
Epoch 9915/10000
32/32 [==============================] - 0s 4ms/step - loss: 1.9403 - accuracy: 0.3303 - val_loss: 4.0380 - val_accuracy: 0.2545
Epoch 9916/10000
32/32 [==============================] - 0s 4ms/step - loss: 1.9002 - accuracy: 0.3211 - val_loss: 4.0252 - val_accuracy: 0.2455
Epoch 9917/10000
32/32 [==============================] - 0s 4ms/step - loss: 1.9586 - accuracy: 0.2936 - val_loss: 4.0518 - val_accuracy: 0.2545
Epoch 9918/10000
32/32 [==============================] - 0s 4ms/step - loss: 1.9368 - accuracy: 0.3096 - val_loss: 4.0451 - val_accuracy: 0.2455
Epoch 9919/10000
32/32 [==============================] - 0s 4ms/step - loss: 1.9429 - accuracy: 0.3440 - val_loss: 4.0445 - val_accuracy: 0.2636
Epoch 9920/10000
32/32 [==============================] - 0s 4ms/step - loss: 1.9300 - accuracy: 0.3142 - val_loss: 4.0382 - val_accuracy: 0.

<H2> Part 3b: Loading a pre-trained Model </H2>

If you have ever run cell 3a), you will load the model that you have trained. Otherwise, you will load a pre-trained model that is shipped with this Jupyter notebook.

In [18]:
#This is code cell 3b) that loads a pre-trained model. 

model = tf.keras.models.load_model('saved_model/my_model')

print("model is loaded")

model is loaded


<H2> Part 4: Evaluation </H2>

This part calculates the confusion matrix and accuracy_score of the model.

In [19]:
# Restore the weights
import numpy as np
import pandas as pd

dataset = pd.read_csv('training_data/horse_data_train_test.csv')
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)


import tensorflow as tf
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Predicting the Test set results
y_pred = model.predict( X_test )  #y_pred = new_model.predict( sc.fit_transform(X_test) ) 
y_pred = np.argmax(y_pred, axis = 1)

In [20]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, (y_pred+1))
print("cm=", cm)
print("accuracy_score=", accuracy_score(y_test, (y_pred+1)))

cm= [[0 1 2 0 0 1 0 0 0 0 0 1 0 0]
 [2 6 0 0 0 3 0 0 1 0 0 0 0 1]
 [3 1 3 0 1 0 0 1 0 0 0 0 0 0]
 [0 2 2 2 0 2 0 0 0 0 0 0 0 0]
 [0 0 1 0 2 0 0 0 0 0 0 0 0 1]
 [2 1 0 0 0 2 0 0 0 1 0 0 0 0]
 [0 1 2 1 0 2 0 0 0 0 0 0 0 0]
 [0 0 1 1 2 0 0 1 1 1 0 1 0 1]
 [0 0 1 1 0 1 1 4 0 1 0 0 0 1]
 [0 1 0 0 3 0 0 2 0 1 0 0 1 0]
 [1 0 1 0 1 1 0 2 0 3 1 1 1 0]
 [0 0 1 1 2 1 0 0 0 0 0 2 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 7 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 5]]
accuracy_score= 0.2909090909090909


<H2> Part 5: Visualization of the model by Tensorboard </H2>

By making use of the powerful visual tools given by Tensorboard, we can tune the hyper parameters of the model and visualize the results easily. If the epoch_accuracy is increasing steadily with epoch for the validation set, we could be confident that the model is on track. 

If we are not satisfy with the performaces shown in the Tensorboard, we can go back to refine the data processing, model building and hyper parameters and check on Tensorboard again iteratively.

In [21]:
print("Calling tensorboard...")

%tensorboard --logdir logs


Calling tensorboard...


Reusing TensorBoard on port 6006 (pid 8984), started 2 days, 9:14:49 ago. (Use '!kill 8984' to kill it.)

<H2> Part 6: Prediction </H2>

We need to create a new csv file that contains the data of a new race in the folder new_data. Then we run the following two cells to get the results.

In [9]:
#This is the csv containing the data of a new race:

CURRENT_RACE_DATA = 'new_data/horse_data_20210922_race3.csv'

In [24]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

PATH_TRAINING_DATA = 'training_data/horse_data_train_test.csv'

#training data
dataset = pd.read_csv(PATH_TRAINING_DATA)
X = dataset.iloc[:, :-1].values
Y = dataset.iloc[:, -1].values

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0, shuffle=False)

sc = StandardScaler()
X_train = sc.fit_transform(X_train)

#live data
dataset = pd.read_csv(CURRENT_RACE_DATA)
X_live = dataset.iloc[:, :-1].values

#use "transform" for live data
X_live = sc.transform(X_live)

#loading pretrained model
model = tf.keras.models.load_model('saved_model/my_model')

# Predicting the Test set results
y_pred = model.predict(X_live)  

#print("y_pred = " + str(y_pred))

# Predicting the finishing position
y_pred_finishing = np.argmax(y_pred, axis = 1) 
print("Expected finishing positions=", y_pred_finishing+1)

print("The real finishing positions= [ 1  6  4  2 12 10  8  3  7  5 11  9 13 14] ")


Expected finishing positions= [ 3  2  1  2  5  2  3 10  3  5  2  5 13 14]
The real finishing positions= [ 1  6  4  2 12 10  8  3  7  5 11  9 13 14] 
